# Perspective Transformation

In [1]:
import cv2, os
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# 1) 영상 불러오기

# img_path = 'money.jpg'  # 소스랑 이미지가 같은 폴더
img_path = './images/money.jpg'
filename, ext = os.path.splitext(os.path.basename(img_path))
ori_img = cv2.imread(img_path)     # 불러온 영상을 ori_img라는 변수에 넣어준다

# src라는 변수에 왼쪽위, 오른쪽위, 오른쪽아래, 왼쪽아래 순서로 각 점의 좌표를 넣는다
src = []         # 전역변수 지정하기

In [3]:
# 2) Mouse Callback Handler

def mouse_handler(event, x, y, flags, param):       # x, y : 마우스의 좌표
    if event == cv2.EVENT_LBUTTONUP:                # 왼쪽 마우스 버튼이 올라왔을 때 이벤트 발생 (누른 다음에 떼었을 때만)
        img = ori_img.copy()
        
        src.append([x, y])                          # src라는 전역변수에 x, y를 저장한다
        
        for xx, yy in src:
            cv2.circle(img, center=(xx, yy), radius=5, color=(0, 255, 0), thickness=-1,     # 클릭한 위치마다 초록색 점을 생성해준다
                      lineType=cv2.LINE_AA)
            
        cv2.imshow('img', img)


        
# 3) Perspective Transformation : 입력 4개, 출력 4개의 점이 필요하다

        if len(src) == 4:                                   # src에 들어가 있는 점이 4개여야만 perspective transform 실행이 가능하다
            src_np = np.array(src, dtype=np.float32)        # src에 있는 점들을 모두 32bit의 실수형으로 바꿔주고, src_np에 넣어준다
            
            # 만들어낸 영상의 가로길이, 세로길이를 계산하는 부분이다
            # width --> 영상의 위쪽 너비(np.linalg.norm()함수로 구현)와 아래쪽 너비 중에서 더 긴(max()함수로 구현) 너비를 쓰겠다
            # height --> 영상의 왼쪽 높이(np.linalg.norm()함수로 구현)와 오른쪽 높이 중에서 더 긴(max()함수로 구현) 높이를 쓰겠다
            width = max(np.linalg.norm(src_np[0] - src_np[1]), np.linalg.norm(src_np[2] - src_np[3]))          # np.linalg.norm(a-b) : a점과 b점 사이의 거리를 구한다
            height = max(np.linalg.norm(src_np[0] - src_np[3]), np.linalg.norm(src_np[1] - src_np[2]))
            
            # 만들어낼 영상은 직사각형이 될 것이므로, 만들어낼 영상의 좌표를 정의한다
            dst_np = np.array([
                [0,0],                # 왼쪽 위의 좌표
                [width, 0],           # 오른쪽 위의 좌표
                [width, height],      # 오른쪽 아래의 좌표
                [0, height]           # 왼쪽 아래의 좌표
            ], dtype=np.float32)
            
            # Perspective Transform이 실질적으로 일어나기 시작하는 부분이다
            # M --> Transformation Matrix. 영상을 어떻게 변형해야 하는지를 알려주는 행렬
            M = cv2.getPerspectiveTransform(src=src_np, dst=dst_np)             # Perspective Transform 행렬(Mask)을 구한다. 구해놓은 src_np와 dst_np를 넣어준다
            result = cv2.warpPerspective(ori_img, M=M, dsize=(width, height))   # '원본 영상을 M과 같은 행렬의 형태로 변환시켜라. 사이즈는 위에서 구한 width, height로 하여라'라는 의미
            
            cv2.imshow('result', result)
            cv2.imwrite('Perspective_img.jpg', result)

In [4]:
# 4) 출력하기

cv2.namedWindow('img')                            # 출력 윈도우의 이름을 지정한다 (아마도 마우스 콜백을 위해서 정의하는 것으로 보임)
cv2.setMouseCallback('img', mouse_handler)        # 특정 윈도우에 마우스 콜백을 지정한다

cv2.imshow('img', ori_img)                        # 'img'라는 윈도우 이름의 ori_img를 화면에 띄운다
cv2.waitKey(0)                                    # 입력 전까지 무한대기 (0)
cv2.destroyAllWindows()